In [9]:
# -*- coding: utf-8 -*-
"""
This script is modified from the nominatim python notebook to use the GC API to geocode 
sources without lat/lons.

The results seem to be less successful than with nominatim.
"""

import requests
import json
import pandas as pd
import time
import os

In [4]:
from dotenv import load_dotenv

In [5]:
load_dotenv()
gc_key = os.environ.get("GC_API_KEY")

In [17]:

params = {'qstr':'4373 Main St Vancouver BC V5V3R1'}

headers = {'user_key': gc_key}
url='https://national-address-register-statcan.api.canada.ca:443/v1/addresses/search'

JSONS=[]
JSONS_CITIES=[]

In [13]:
headers

{'user_key': '3fe6ce91c9fa569864d885a315721c13',
 'qstr': '4373 Main St Vancouver BC V5V3R1'}

In [22]:

#     query = reqs[2]
#     query2 = reqs2[2]
#     query3 = reqs3[2]

coords = requests.get(url, params=params, headers=headers)

print(coords.status_code)
# if coords:
#     resp = coords.json()

#     print(resp)

#     JSONS.append(resp)

#     if resp!=[]:
#         print("response: " + resp['display_name'])


204


In [5]:
df = pd.read_csv("combined.csv", low_memory=False, dtype="str")

In [6]:
# filter to just those without lat/lon
df = df[df.latitude.isnull()]
len(df)

150

In [7]:
# we try three requests: facility name, city and street address
df['nom_request'] = df['facility_name'] + ', ' + 'Canada'
df['nom_request_2'] = df['city'] + ', ' + df['province'] + ', ' + 'Canada'
df['nom_request_3'] = df['street_no'] + ', ' + df['street_name'] + ', ' + df['postal_code'] + ', '+ df['city'] + ', ' + df.province + ', ' + 'Canada'

df.fillna('', inplace=True)
facility_name = df['facility_name']
street_no = df['street_no']
street_name = df['street_name']
city = df['city']
postal_code = df['postal_code']
province = df['province']
comma_space = ', '
can = 'Canada'

df['nom_request'] = facility_name + comma_space + city + comma_space + province + comma_space + can
df['nom_request_2'] = city + comma_space + province + comma_space + can
df['nom_request_3'] = street_no + ' ' + street_name + comma_space + city + comma_space + province + comma_space + can

# df['geo_source'] = df.geo_source.fillna('')

# add columns with nom 
reqs=list(df['nom_request'])
reqs2=list(df['nom_request_2'])
reqs3=list(df['nom_request_3'])

In [ ]:
reqs

In [ ]:
df = df[:8]

In [ ]:
len(df)

In [ ]:
# geo_source=list(df['geo_source'])

for i in range(len(reqs)):
# for i in range(5):
    query = reqs[i]
    query2 = reqs2[i]
    query3 = reqs3[i]
    
#     source=geo_source[i]
    
#     if (source!=''):
#         JSONS.append([])
#         #JSONS_CITIES.append([])
#     else:
        
    params={'q':query3,
            'addressdetails':'1',
            'format':'json',
            'email':'sam.lumley@statcan.gc.ca'}
    time.sleep(6) # 6
    coords=requests.get(url, 
                    params=params, headers=headers)
    resp=coords.json()
    if len(resp)>0:
        resp=resp[0]
    JSONS.append(resp)
    print(i)
    print("query: " + query)
    if resp!=[]:
        print("response: " + resp['display_name'])
            
        #repeat for just city and province
#        params={'q':query2,
#                'addressdetails':'1',
#                'format':'json',
#                'email':'joseph.kuchar@canada.ca'}
#        time.sleep(6)
#        coords=requests.get(url, 
#                        params=params,headers=headers)
#        resp=coords.json()
#        if len(resp)>0:
#            resp=resp[0]
#        JSONS_CITIES.append(resp)
#        print(i)
#        print(query2)
#        if resp!=[]:
#            print(resp['display_name'])

In [ ]:
with open('Nominatim.json', 'w', encoding='utf-8') as f:
    json.dump(JSONS, f, ensure_ascii=False, indent=4) 

In [ ]:
# read json request results into our dataframe

with open('Nominatim.json', 'r', encoding='utf-8') as f:
    JSONS=json.load(f)    
        
LATS=[]
LONS=[]
ADDR=[]
CITY=[]
PROV=[]
NAME=[]
TYPE=[]
COUNTRY=[]
POST=[]
CITY=[]
CLASS=[]
for element in JSONS:
    if element==[]:
        LATS.append('')
        LONS.append('')
        NAME.append('')
        ADDR.append('')
        PROV.append('')
        TYPE.append('')
        COUNTRY.append('')
        POST.append('')
        CITY.append('')
        CLASS.append('')
    else:
        
        print(element['class'])
        # we might only use results that are classed by osm as "amenity" 
        # For now we will use everything
        
#         if (element['address']['country_code']=='ca') and ('amenity' in element['address'].keys()): 
#         if (element['address']['country_code']=='ca') and ((element['class'] in ['amenity', 'place', 'building']) ):
            
        if (element['address']['country_code']=='ca'):
            
            COUNTRY.append(element['address']['country_code'])
        
        
            LATS.append(element['lat'])
            LONS.append(element['lon'])
            ADDR.append(element['display_name'])
            if 'type' in element.keys():
                TYPE.append(element['type'])
            else:
                TYPE.append('')
            if 'class' in element.keys():
                CLASS.append(element['class'])
            else:
                CLASS.append('')
            
            if 'amenity' in element.keys():
                NAME.append(element['address']['amenity'])
            else:
                NAME.append(element['display_name'])
            if 'state' in element.keys():
                PROV.append(element['address']['state'])
            else:
                PROV.append('')
            if 'postcode' in element['address']:
                POST.append(element['address']['postcode'])
            else:
                POST.append('')
            if 'city' in  element['address']:
                CITY.append(element['address']['city'])
            else:
                CITY.append('')
                
        else:
            LATS.append('')
            LONS.append('')
            NAME.append('')
            ADDR.append('')
            PROV.append('')
            CLASS.append('')
            TYPE.append('')
            COUNTRY.append('')
            CITY.append('')
            

            
df['osm_address']=ADDR
df['osm_name']=NAME
df['osm_lat']=LATS
df['osm_lon']=LONS
df['osm_prov']=PROV
df['osm_country']=COUNTRY
df['osm_class']=CLASS
df['osm_type']=TYPE
df['osm_city']=CITY


In [ ]:
len(df)

In [ ]:
df.to_csv("ODHFv2_Geocoded_OSM_15-02-2022.csv", index=False)

In [ ]:
%env